In [1]:
import os
import itertools
from keras.callbacks import EarlyStopping
from sklearn.metrics import f1_score
import tools

training_data, training_label, validation_data, validation_label, validation_cate_label = tools.get_data()

kernel_size = [7]
num_layers = [8]
batch_size = [30]
learning_rate = [0.0001]
overflow_model = 0

def run(model, bs, path):
    fold=1
    
    for X_train, Y_train, X_val, Y_val, val_cat in zip(training_data,
                                                       training_label,
                                                       validation_data,
                                                       validation_label,
                                                       validation_cate_label):
        print("Fold "+str(fold))
        inner_path = path+"/fold_"+str(fold)
        if not os.path.exists(inner_path):
            os.makedirs(inner_path)
        
        early_stop = EarlyStopping(patience=20)
        history = model.fit(x = X_train, 
                            y = Y_train,
                            epochs=80,
                            validation_data=(X_val, Y_val),
                            callbacks=[early_stop],
                            batch_size=bs, 
                            verbose=0)
        evaluation = model.evaluate(x = X_val, y = Y_val)
        validation_prediction = model.predict_classes(X_val, batch_size=bs)
        score = f1_score(val_cat, validation_prediction, average=None)
        
        tools.show_plot(inner_path, history)
        tools.write_file(inner_path+"/readme.txt", evaluation, score, model)
        fold = fold + 1

## main()
base_path = "5_fold/"
if not os.path.exists(base_path):
    os.makedirs(base_path)
index=1
with open(base_path+"Cartesian_product.txt", "w+") as writer:
    for ks, num_layer, bs, lr in itertools.product(kernel_size, num_layers, batch_size, learning_rate):
        writer.write(str(index)+":"+"\n")
        writer.write('ks={}, num_layer={}, bs={}, lr={}'.format(ks, num_layer, bs, lr)+"\n")
        index = index + 1
    del index

index=1
for ks, num_layer, bs, lr in itertools.product(kernel_size, num_layers, batch_size, learning_rate):
    print("Index "+str(index))
    path = base_path+str(index)
    if not os.path.exists(path):
        os.makedirs(path)
    
    model = tools.create_model(lr, bs, ks, num_layer)
    if model:
        run(model, bs, path)
    else:
        overflow_model = overflow_model + 1
    del model
    index = index + 1

print("Training finished! Overflow model: ", overflow_model)

Using TensorFlow backend.


Train Data: (10151, 9000, 1)
Train Label:  (10151, 4)
KFold(n_splits=5, random_state=None, shuffle=True)
trian: [    0     1     2 ... 10146 10149 10150] len 8120 test: [    8    11    12 ... 10136 10147 10148] len 2031
trian: [    0     1     2 ... 10146 10147 10148] len 8121 test: [    5     7     9 ... 10145 10149 10150] len 2030
trian: [    0     1     5 ... 10148 10149 10150] len 8121 test: [    2     3     4 ... 10140 10141 10143] len 2030
trian: [    1     2     3 ... 10148 10149 10150] len 8121 test: [    0     6    10 ... 10135 10142 10146] len 2030
trian: [    0     2     3 ... 10148 10149 10150] len 8121 test: [    1    13    18 ... 10118 10120 10138] len 2030
Index 1
Fold 1
2031/2031 [==============================] - 1s 432us/step
Fold 2
2030/2030 [==============================] - 1s 419us/step
Fold 3
2030/2030 [==============================] - 1s 400us/step
Fold 4
2030/2030 [==============================] - 1s 399us/step
Fold 5
2030/2030 [==============================